In [1]:
import sqlite3
import pandas as pd
import numpy as np
import datetime

In [2]:
from utils.station_info import town_and_station
from data_integration.integrating_forecast import retrieve_forecast_from_sql, sample_forecast_with_given_deltaday, sample_forecast_at_given_time
from data_integration.integrating_weather_data import get_entire_day_observation

In [3]:
sql_db_fn = './realtime/realtime_data/realtime.db'
station_and_town = {v:k for k, v in town_and_station.items()}

In [10]:
date_str = '2024-09-19'
station = '臺中電廠'

In [5]:
def get_forecast(station, date_str, sql_db_fn=sql_db_fn):
    start_date = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    end_date = start_date + datetime.timedelta(days=1)
    sample_time = start_date + datetime.timedelta(hours=19)
    forecast_df = retrieve_forecast_from_sql(station_and_town[station], sql_db_fn,
                                             start_date=start_date, end_date=end_date)
    forecast_df = sample_forecast_with_given_deltaday(forecast_df, deltaday=1)
    forecast_df = sample_forecast_at_given_time(forecast_df, sample_time)
    return forecast_df

In [6]:
def get_observation_df(station, date_str, sql_db_fn=sql_db_fn):
    sql_df = get_entire_day_observation(datetime.datetime.strptime(date_str, '%Y-%m-%d'), station, sql_db_fn)
    sql_df['Time'] = pd.to_datetime(sql_df['Time'])
    obs_df = sql_df.rename(columns={'Time':'目標時間'})
    return obs_df

In [11]:
forecast_df = get_forecast(station, date_str)
obs_df = get_observation_df(station, date_str)
df = pd.merge(obs_df, forecast_df, on='目標時間')
df

,Station,目標時間,Temperature,Weather,Wind_Direction,Wind_Speed,Gust_Wind,Humidity,Pressure,Rainfall,Sunlight,鄉鎮,預測時間,天氣狀況,溫度,降水機率,相對溼度,風速,風向
0,臺中電廠,2024-09-19 03:00:00,27.3,多雲,南南東,1.4,None,90,None,0.0,None,臺中市龍井區,2024-09-18 19:01:02,晴,28,0.1,85,1,東南風
1,臺中電廠,2024-09-19 06:00:00,27.4,多雲,南,1.2,None,89,None,0.0,None,臺中市龍井區,2024-09-18 19:01:02,晴,27,0.1,84,2,西南風


In [ ]:
pd.merge(sql_df, forecast_df, left_on='Time', right_on)

In [12]:
forecast_df

,鄉鎮,預測時間,目標時間,天氣狀況,溫度,降水機率,相對溼度,風速,風向
24,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 00:00:00,多雲,28,0.1,83,2,東南風
25,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 03:00:00,晴,28,0.1,85,1,東南風
26,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 06:00:00,晴,27,0.1,84,2,西南風
27,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 09:00:00,晴,30,0.1,72,3,西南風
28,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 12:00:00,多雲,32,0.1,69,4,西南風
29,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 15:00:00,多雲,32,0.1,78,3,西南風
30,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 18:00:00,晴,30,0.1,83,2,偏南風
31,臺中市龍井區,2024-09-18 19:01:02,2024-09-19 21:00:00,多雲,29,0.1,85,1,偏東風
